In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1748258165689_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sc

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-0>

In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import monotonically_increasing_id
import time
import random

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Leer las tablas de Glue Catalog
df_standard = spark.table("proyecto1db.data_standard")
df_prepared_selected = spark.table("proyecto1db.data_prepared_selected")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
print((df_standard.count(),len(df_standard.columns)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(3043, 21)

In [6]:
print((df_prepared_selected.count(),len(df_prepared_selected.columns)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(3043, 19)

In [7]:
# Obtener lista única de países
countries = [row['country'] for row in df_standard.select("country").distinct().collect()]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# Particionar países aleatoriamente en entrenamiento y prueba (80/20)
semilla = 42
random.seed(semilla)

train_size = int(len(countries) * 0.8)
train_countries = random.sample(countries, train_size)
test_countries = list(set(countries) - set(train_countries))

# Verificación
print(f"Paises de entrenamiento: {len(train_countries)}")
print(f"Paises de prueba: {len(test_countries)}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Paises de entrenamiento: 143
Paises de prueba: 36

In [9]:
train_countries

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Liberia', 'Mauritius', 'Peru', 'Greece', 'Costa Rica', 'Seychelles', 'Tonga', 'Poland', 'Hungary', 'Lebanon', 'Gabon', 'Slovakia', 'Croatia', 'China', 'Slovenia', 'United Arab Emirates', "Lao People's Democratic Republic", 'Saudi Arabia', 'Mauritania', 'El Salvador', 'Eritrea', 'Lithuania', 'Dominican Republic', 'Sri Lanka', 'United Kingdom of Great Britain and Northern Ireland', 'Kuwait', 'Bolivia (Plurinational State of)', 'Republic of North Macedonia', 'Turkey', 'Togo', 'Guinea', 'Latvia', 'Gambia', 'Rwanda', 'South Africa', 'Jamaica', 'Bosnia and Herzegovina', 'Namibia', 'Burkina Faso', 'Viet Nam', 'Mozambique', 'Saint Lucia', 'Singapore', 'Niger', 'Burundi', "Côte d'Ivoire", 'Haiti', 'Zambia', 'United Republic of Tanzania', 'Belize', 'Nigeria', 'Mexico', 'Nicaragua', 'Azerbaijan', 'Mali', 'Colombia', 'Malawi', 'Norway', 'Belgium', 'Zimbabwe', 'Ecuador', 'Ireland', 'Chile', 'Republic of Moldova', 'Brazil', 'Cyprus', 'Cambodia', 'India', 'Samoa', 'Sao Tome and Principe', 'Sierra L

In [10]:
test_countries

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Argentina', 'Venezuela (Bolivarian Republic of)', 'Iraq', 'Antigua and Barbuda', 'Central African Republic', 'Comoros', 'Belarus', 'Iran (Islamic Republic of)', 'Honduras', 'Botswana', 'Ghana', 'Uruguay', 'Papua New Guinea', 'Philippines', 'Thailand', 'Luxembourg', 'Denmark', 'Democratic Republic of the Congo', 'Paraguay', 'Italy', 'Bhutan', 'Bulgaria', 'Georgia', 'Nepal', 'Finland', 'New Zealand', 'Solomon Islands', 'Netherlands', 'Mongolia', 'Fiji', 'Iceland', 'Cameroon', 'Australia', 'Benin', 'Sweden', 'Congo']

In [34]:
df_standard = df_standard.withColumn("id", monotonically_increasing_id())
df_prepared_selected = df_prepared_selected.withColumn("id", monotonically_increasing_id())

# Ahora puedes hacer join por `id`
df_prepared_selected = df_prepared_selected.join(
    df_standard.select("id", "country", "region"),
    on="id",
    how="left"
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
df_prepared_selected.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+-------------------+--------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-----------+------+-------+------+
| id|               year|             une_pop|            une_gni|             gghe-d|           che_gdp|      une_edu_spend|        basic_water|            doctors|                bmi|   age5-19thinness|     age5-19obesity|            alcohol|          hepatitis|            measles|              polio|         diphtheria|             une_hiv|life_expect|africa|country|region|
+---+-------------------+--------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+----------------

In [36]:
print((df_prepared_selected.count(),len(df_prepared_selected.columns)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(3043, 22)

In [37]:
# Crear conjuntos de entrenamiento y prueba
train_df = df_prepared_selected.filter(col("country").isin([str(c) for c in train_countries]))
test_df = df_prepared_selected.filter(col("country").isin([str(c) for c in test_countries]))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
train_df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+-------------------+--------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-----------+------+-------+------+
| id|               year|             une_pop|            une_gni|             gghe-d|           che_gdp|      une_edu_spend|        basic_water|            doctors|                bmi|   age5-19thinness|     age5-19obesity|            alcohol|          hepatitis|            measles|              polio|         diphtheria|             une_hiv|life_expect|africa|country|region|
+---+-------------------+--------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+----------------

In [39]:
print((train_df.count(),len(train_df.columns)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(2431, 22)

In [40]:
print((test_df.count(),len(test_df.columns)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(612, 22)

In [45]:
train_df = train_df.drop('id')
test_df = test_df.drop('id')
df_prepared_selected = df_prepared_selected.drop('id')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
# Definir columnas predictoras (excluyendo la variable objetivo y columnas identificadoras)
target_col = "life_expect"
id_cols = ["country", "region", "year"]
feature_cols = [c for c in df_prepared_selected.columns if c not in id_cols + [target_col]]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
# Ensamblar características en un solo vector
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
train_df = assembler.transform(train_df).select("features", target_col, "country", "region")
test_df = assembler.transform(test_df).select("features", target_col, "country", "region")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
train_df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+-------+------+
|            features|life_expect|country|region|
+--------------------+-----------+-------+------+
|[-0.1191951744334...|    47.3373| Angola|Africa|
|[-0.1132293737888...|   48.19789| Angola|Africa|
|[-0.1060607469518...|   49.42569| Angola|Africa|
|[-0.0982694597124...|   50.50266| Angola|Africa|
|[-0.0907255082841...|   51.52863| Angola|Africa|
+--------------------+-----------+-------+------+
only showing top 5 rows

In [50]:
# Iniciar el temporizador
start_time = time.time()

# Entrenar modelo de RandomForestRegressor
rf = RandomForestRegressor(
    featuresCol="features", 
    labelCol=target_col,
    numTrees=144,
    maxDepth=20,
    minInstancesPerNode=5,
    minInfoGain=0.0,
    maxBins=32,
    seed=semilla,
    featureSubsetStrategy="log2",
    bootstrap=False
)

model = rf.fit(train_df)

# Finalizar el temporizador
end_time = time.time()

# Imprimir duración del entrenamiento
duration = end_time - start_time
print(f"Tiempo de entrenamiento del modelo: {duration:.2f} segundos")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Tiempo de entrenamiento del modelo: 34.58 segundos

In [51]:
# Hacer predicciones
predictions = model.transform(test_df)

# Evaluación del modelo
evaluator = RegressionEvaluator(labelCol=target_col, predictionCol="prediction", metricName="mse")
mse = evaluator.evaluate(predictions)
mae = evaluator.setMetricName("mae").evaluate(predictions)

print("MSE:", mse)
print("MAE:", mae)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

MSE: 32.754939898875115
MAE: 4.199028801678624

In [52]:
predictions.select("country", "region", "prediction", target_col).show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------+------------------+------------------+
|     country|region|        prediction|       life_expect|
+------------+------+------------------+------------------+
|South Africa|Africa| 60.56887844146823|          58.98315|
|South Africa|Africa| 61.07531113996359|          57.90976|
|South Africa|Africa| 60.73301004907957|56.844190000000005|
|South Africa|Africa|61.403959023644184|          55.82792|
|South Africa|Africa| 61.34666747337964|54.923559999999995|
+------------+------+------------------+------------------+
only showing top 5 rows

In [53]:
# Guardar predicciones con columnas relevantes
predictions.select("country", "region", "prediction", target_col) \
    .write.mode("overwrite") \
    .parquet("s3://aarangog2proyectointegrador/zona refined/predicciones_rf")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
model.write().overwrite().save("s3://aarangog2proyectointegrador/zona refined/modelo_rf")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…